# Subgradient projection method for basis pursuit

Consider the basis pursuit problem:
$$
\begin{aligned}
&\text{minimize } &&\|x\|_1 = \sum_{i=1}^n |x_i|,\\
&\text{subject to } && Ax=b.
\end{aligned}
$$
We assume that $A\in\mathbb{R}^{m\times n}$ has full row rank, $m<n$, $b\in \text{range }A \subseteq \mathbb{R}^m$.  A code snippet below generates a random instance of such a problem:

In [1]:
import numpy as np

def random_instance(m,n):
    A = np.random.rand(m,n)
    b = A.sum(axis=1)
    return (A,b)

We now provide a prototypical implementation of the subgradient projection algorithm for this problem:

In [20]:
from scipy.linalg import cho_factor, cho_solve

def subgradient(x):
    # subgradient of 1-norm
    return np.sign(x)

def project(y,A,b,AAT_fact):
    # Projection of y onto the affine manifold {x: Ax=b}
    return y-A.T@(cho_solve(AAT_fact, A@y-b))

def subgradient_method(x0,A,b):
    xk       = x0.copy()
    x_best   = xk.copy()
    f_best   = float('inf')
    # we prefactorize AA^T, since we are solving this system
    # at every iteration
    AAT_fact = cho_factor(A@A.T)
    k_max    = 1000000
    k_print  = 10000
    x_diff   = 1.0E-06
    for k in range(k_max):
        # get a subgradient
        gk = subgradient(xk)
        # Since subradients are not necessarily descent directions,
        # we cannot use linesearch. Instead, we use the rule
        # where alpha_k->0 but sum(alpha_k)=infinity.
        alphak = 1.0/(1+k)
        # Project the step onto the feasible set
        xk1 = project(xk-alphak*gk,A,b,AAT_fact)
        # Compute the new objective and possibly update
        # the best so far
        fk1 = np.linalg.norm(xk1,ord=1)
        if fk1 < f_best:
            x_best = xk1.copy()
            f_best = fk1
        # Print messages every k_print iterations
        if k%k_print == 0:
            print("Iter: %6d, fk: %e" % (k,fk1))
        # We stop when the method stops making progress.
        # This is not a good practice; one should instead 
        # check the optimality conditions in general.
        if np.linalg.norm(xk-xk1,np.inf) < x_diff:
            break
        xk = xk1
    return x_best,k

Let us test it on some small problems.

In [22]:
A,b=random_instance(5,20)
x,k =subgradient_method(np.zeros(A.shape[1]),A,b)
f   = np.linalg.norm(x,ord=1)
print("Iter: %6d, f : %e" % (k,f))

Iter:      0, fk: 1.822785e+01
Iter:  10000, fk: 1.477722e+01
Iter:  20000, fk: 1.475342e+01
Iter:  30000, fk: 1.473940e+01
Iter:  40000, fk: 1.472954e+01
Iter:  50000, fk: 1.472187e+01
Iter:  60000, fk: 1.471562e+01
Iter:  70000, fk: 1.471032e+01
Iter:  80000, fk: 1.470573e+01
Iter:  90000, fk: 1.470169e+01
Iter: 100000, fk: 1.469809e+01
Iter: 110000, fk: 1.469481e+01
Iter: 120000, fk: 1.469184e+01
Iter: 130000, fk: 1.468908e+01
Iter: 140000, fk: 1.468655e+01
Iter: 150000, fk: 1.468419e+01
Iter: 160000, fk: 1.468197e+01
Iter: 170000, fk: 1.467989e+01
Iter: 180000, fk: 1.467793e+01
Iter: 190000, fk: 1.467607e+01
Iter: 200000, fk: 1.467432e+01
Iter: 210000, fk: 1.467265e+01
Iter: 220000, fk: 1.467105e+01
Iter: 230000, fk: 1.466952e+01
Iter: 240000, fk: 1.466807e+01
Iter: 250000, fk: 1.466667e+01
Iter: 260000, fk: 1.466532e+01
Iter: 270000, fk: 1.466403e+01
Iter: 280000, fk: 1.466278e+01
Iter: 290000, fk: 1.466158e+01
Iter: 300000, fk: 1.466042e+01
Iter: 310000, fk: 1.465930e+01
Iter: 32